<a href="https://colab.research.google.com/github/rprimi/b5_nlp/blob/main/m2_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pacakges <- c("tidyverse", "keras", "psych")
install.packages(pacakges)

install.packages("readr")
install.packages("zip")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [10]:
library(tidyverse)
library(keras)
library(psych)
library(readr)
library(zip)



Attaching package: ‘zip’


The following objects are masked from ‘package:utils’:

    unzip, zip




In [4]:
load("base_bfi_nlp.RData")

In [ ]:
samples <- db_textos_splited$texto_dividido

tokenizer <- text_tokenizer(
    lower = TRUE )  %>%
    fit_text_tokenizer(samples)

word_index <- tokenizer$word_index

sequences <- texts_to_sequences(tokenizer, samples)

cat("Found", length(word_index), "unique tokens.\n")


In [ ]:
library(zip)

# Download the zip file
download.file("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s600.zip", destfile = "file.zip")

# Read the CSV file from the zip file
glove_pt_600 <- read.zip("file.zip", "file.csv")



In [ ]:
 load("nilc_wv_glove_600d.RData")
 ls()


 glove_pt_600 = data.table::fread(
    file = "~/Downloads/glove_s600.txt",
    encoding = 'UTF-8',
    # nrows = 100,
    skip = 1
  )

  db_textos_embeddings <- db_textos_splited %>%
    unnest_tokens(word, texto_dividido) %>%
    left_join(glove_pt_600, by =c("word"="V1")) %>%
    select(-word) %>%
    group_by(id, id_divisao) %>%
    dplyr::summarise(
    across(V2:V601, mean, na.rm=TRUE)
)